In [5]:
# === Minimal one-cell VDM debug: load, instantiate, smoke test, summary ===
import os, sys, importlib.util, types
import torch
import torch.nn as nn
from torchinfo import summary

# -------------------------------------------------
# 0) Paths (네가 말한 실제 경로 기준)
# -------------------------------------------------
PROJECT_ROOT = "/home/mingyeong/GAL2DM_ASIM_VDM"
SRC_DIR      = os.path.join(PROJECT_ROOT, "src")

if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# 이제 src/ 아래의 model.py, model_tools.py 를
# 그냥 top-level 모듈처럼 import 가능해짐.
from model import VDM   # <-- 방금 만든 clean VDM 클래스
import model_tools      # <-- 필요하면 직접 참조 가능

print("✅ Imported VDM from", os.path.join(SRC_DIR, "model.py"))

# -------------------------------------------------
# 1) mltools 패키지 컨텍스트 안에서 vdm_model / model_tools 로드
#    (vdm_model.py 내부의 `from mltools.models.model_tools import ...` 대응)
# -------------------------------------------------
def load_vdm_symbol(vdm_path, modeltools_path, symbol):
    # (1) fake 패키지: mltools, mltools.models
    if "mltools" not in sys.modules:
        mltools_pkg = types.ModuleType("mltools")
        mltools_pkg.__path__ = []
        sys.modules["mltools"] = mltools_pkg

    if "mltools.models" not in sys.modules:
        models_pkg = types.ModuleType("mltools.models")
        models_pkg.__path__ = []
        sys.modules["mltools.models"] = models_pkg

    # (2) mltools.models.model_tools 로 로드
    if "mltools.models.model_tools" not in sys.modules:
        t_spec = importlib.util.spec_from_file_location(
            "mltools.models.model_tools", modeltools_path
        )
        t_mod = importlib.util.module_from_spec(t_spec)
        t_mod.__package__ = "mltools.models"
        sys.modules["mltools.models.model_tools"] = t_mod
        t_spec.loader.exec_module(t_mod)  # type: ignore

    # (3) mltools.models.vdm_model 로 로드
    m_spec = importlib.util.spec_from_file_location(
        "mltools.models.vdm_model", vdm_path
    )
    m_mod = importlib.util.module_from_spec(m_spec)
    m_mod.__package__ = "mltools.models"
    sys.modules["mltools.models.vdm_model"] = m_mod
    m_spec.loader.exec_module(m_mod)  # type: ignore

    return getattr(m_mod, symbol)




✅ Imported VDM from /home/mingyeong/GAL2DM_ASIM_VDM/src/model.py


In [6]:
# -------------------------------------------------
# 2) VDM / LightVDM 클래스 로드
# -------------------------------------------------
VDM      = load_vdm_symbol(VDM_PATH, MODELTOOLS_PATH, "VDM")
#LightVDM = load_vdm_symbol(VDM_PATH, MODELTOOLS_PATH, "LightVDM")



In [7]:

# -------------------------------------------------
# 3) Dummy 3D score model 정의
#    - VDM.get_pred_noise에서 호출되는 인터페이스와 호환
#    - forward(zt, t, **kwargs) 형태만 맞추면 됨
# -------------------------------------------------
class DummyScoreNet3D(nn.Module):
    def __init__(self, in_ch: int = 1, hidden: int = 16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv3d(in_ch, hidden, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(hidden, hidden, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(hidden, in_ch, kernel_size=3, padding=1),
        )
        # VDM.sample() 에서 사용하는 shape 정보
        # (C, D, H, W) 형태로 넣어주면 됨
        self.shape = (in_ch, 32, 32, 32)

    def forward(self, zt: torch.Tensor, t: torch.Tensor, **kwargs) -> torch.Tensor:
        # 여기서는 t, kwargs는 무시하고 단순 Conv stack만 수행
        return self.net(zt)



In [8]:

# -------------------------------------------------
# 4) VDM 인스턴스 생성
# -------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

score_model = DummyScoreNet3D(in_ch=1).to(device)
vdm = VDM(
    score_model=score_model,
    noise_schedule="fixed_linear",  # "learned_linear", "learned_nn", "sigmoid" 등으로 바꿔도 됨
    gamma_min=-13.3,
    gamma_max=5.0,
    antithetic_time_sampling=True,
    data_noise=1.0e-3,
).to(device)
vdm.eval()

print("✅ VDM instantiated on", device)



✅ VDM instantiated on cuda


In [10]:
# -------------------------------------------------
# 5) Forward smoke test: get_loss() 체크
#    (⚠️ inference_mode 쓰면 안 됨)
# -------------------------------------------------
x = torch.randn(2, 1, 32, 32, 32, device=device)
loss, metrics = vdm.get_loss(x)   # 여기서는 grad 필요

print("✅ get_loss() OK")
print("   elbo:", float(metrics["elbo"]))
print("   diffusion_loss:", float(metrics["diffusion_loss"]))
print("   latent_loss:", float(metrics["latent_loss"]))
print("   reconstruction_loss:", float(metrics["reconstruction_loss"]))


✅ get_loss() OK
   elbo: 5.821023941040039
   diffusion_loss: 13.24298095703125
   latent_loss: 0.004859061911702156
   reconstruction_loss: -7.426815986633301


In [11]:
# -------------------------------------------------
# Sampling 스모크 테스트
# -------------------------------------------------
with torch.no_grad():
    z_samples = vdm.sample(
        batch_size=1,
        n_sampling_steps=8,
        device=device,
        verbose=False,
    )
print("✅ sample() OK | sample shape:", tuple(z_samples.shape))


✅ sample() OK | sample shape: (1, 1, 32, 32, 32)


In [12]:

# -------------------------------------------------
# 6) Sampling smoke test: sample() 체크
# -------------------------------------------------
with torch.inference_mode():
    z_samples = vdm.sample(
        batch_size=1,
        n_sampling_steps=8,
        device=device,
        verbose=False,
    )
print("✅ sample() OK | sample shape:", tuple(z_samples.shape))



✅ sample() OK | sample shape: (1, 1, 32, 32, 32)


In [14]:
from torchinfo import summary

# 더미 입력 (zt, t)
xt_dummy = torch.randn(1, 1, 32, 32, 32, device=device)
t_dummy  = torch.zeros(1, device=device)  # 그냥 0.0 time

summary(
    vdm.score_model,
    input_data=(xt_dummy, t_dummy),   # ← 튜플로 전달
    device=device,
    depth=2,
    col_names=("input_size", "output_size", "num_params"),
)


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
DummyScoreNet3D                          [1, 1, 32, 32, 32]        [1, 1, 32, 32, 32]        --
├─Sequential: 1-1                        [1, 1, 32, 32, 32]        [1, 1, 32, 32, 32]        --
│    └─Conv3d: 2-1                       [1, 1, 32, 32, 32]        [1, 16, 32, 32, 32]       448
│    └─ReLU: 2-2                         [1, 16, 32, 32, 32]       [1, 16, 32, 32, 32]       --
│    └─Conv3d: 2-3                       [1, 16, 32, 32, 32]       [1, 16, 32, 32, 32]       6,928
│    └─ReLU: 2-4                         [1, 16, 32, 32, 32]       [1, 16, 32, 32, 32]       --
│    └─Conv3d: 2-5                       [1, 16, 32, 32, 32]       [1, 1, 32, 32, 32]        433
Total params: 7,809
Trainable params: 7,809
Non-trainable params: 0
Total mult-adds (M): 255.89
Input size (MB): 0.13
Forward/backward pass size (MB): 8.65
Params size (MB): 0.03
Estimated Total Size (MB): 8.81